In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
import Graph
import Path
import Evolutionary

Testing graph implementation

In [34]:
graph = Graph.get_graph()

In [ ]:
graph.edges

In [ ]:
graph.vertices

In [ ]:
assert len(graph.edges) == len(graph.vertices)

In [9]:
graph.get_neighbours('POLAND')

['BELARUS',
 'CZECH_REPUBLIC',
 'DENMARK',
 'GERMANY',
 'LITHUANIA',
 'RUSSIA',
 'SLOVAKIA',
 'SWEDEN',
 'UKRAINE']

In [ ]:
assert graph.get_neighbours('Czechia') == graph.get_neighbours('Czech_Republic')

In [ ]:
graph.get_neighbours('Czech_Republic')

In [12]:
(date(2020, 5, 14) - date(2020, 5, 7)).days

7

In [10]:
graph.get_stats('POLAND')

array([[   8,    5, 2020,  307,   22],
       [   7,    5, 2020,  309,   17],
       [   6,    5, 2020,  425,   18],
       [   5,    5, 2020,  313,   20],
       [   4,    5, 2020,  318,   14],
       [   3,    5, 2020,  270,   13],
       [   2,    5, 2020,  228,    7],
       [   1,    5, 2020,  237,   20],
       [  30,    4, 2020,  422,   28],
       [  29,    4, 2020,  316,   34],
       [  28,    4, 2020,  285,   27],
       [  27,    4, 2020,  344,   11],
       [  26,    4, 2020,  381,   30],
       [  25,    4, 2020,  381,   40],
       [  24,    4, 2020,  342,   28],
       [  23,    4, 2020,  313,   25],
       [  22,    4, 2020,  263,   21],
       [  21,    4, 2020,  306,   20],
       [  20,    4, 2020,  545,   13],
       [  19,    4, 2020,  363,   15],
       [  18,    4, 2020,  461,   18],
       [  17,    4, 2020,  336,   28],
       [  16,    4, 2020,  380,   23],
       [  15,    4, 2020,  268,   18],
       [  14,    4, 2020,  260,   13],
       [  13,    4, 2020,

In [ ]:
assert np.array_equal(graph.get_stats('Czechia'), graph.get_stats('Czech_Republic'))

In [ ]:
graph.get_stats('Poland')

In [ ]:
assert graph.get_active_cases("Poland", date(2020, 3, 10)) == 17

In [ ]:
assert graph.get_active_cases("Poland", date(2020, 3, 16)) == 122

In [ ]:
assert graph.get_active_cases("Poland", date(2020, 3, 15)) == 102

Testing evolutionary algorithm

In [65]:
start_day = date(2020, 4, 24)
test = Evolutionary.EvolutionaryAlgorithm(graph, start_day, 10, 12, 0.05, 'best', 10)

In [37]:
for i in list(graph.edges.keys()):
    if(len(graph.edges[i]) == 0):
        print(i)

In [66]:
test.start_generation()

In [68]:
for i in test.active_generation:
    print(i.countries)

['MYANMAR', 'LAOS', 'CAMBODIA', 'LAOS', 'CAMBODIA', 'THAILAND', 'INDONESIA', 'MALAYSIA', 'SINGAPORE', 'INDONESIA', 'THAILAND', 'INDONESIA', 'SINGAPORE', 'INDONESIA']
['CANADA', 'FRANCE', 'FIJI', 'TUVALU', 'KIRIBATI', 'JARVIS_ISLAND', 'KIRIBATI', 'UNITED_STATES', 'HAITI', 'NAVASSA_ISLAND', 'JAMAICA', 'HAITI', 'DOMINICAN_REPUBLIC', 'NETHERLANDS']
['SAMOA', 'WALLIS_AND_FUTUNA', 'SAMOA', 'TONGA', 'WALLIS_AND_FUTUNA', 'SAMOA', 'TONGA', 'WALLIS_AND_FUTUNA', 'TOKELAU', 'FRANCE', 'TUVALU', 'KIRIBATI', 'UNITED_STATES', 'JAMAICA']
['GEORGIA', 'RUSSIA', 'AZERBAIJAN', 'ARMENIA', 'TURKEY', 'SYRIA', 'LEBANON', 'SYRIA', 'TURKEY', 'IRAQ', 'SYRIA', 'JORDAN', 'SYRIA', 'JORDAN']
['TURKMENISTAN', 'IRAN', 'AZERBAIJAN', 'ARMENIA', 'AZERBAIJAN', 'RUSSIA', 'NORWAY', 'SWEDEN', 'POLAND', 'DENMARK', 'NORWAY', 'UNITED_KINGDOM', 'DENMARK', 'GERMANY']
['PITCAIRN_ISLANDS', 'FRANCE', 'PITCAIRN_ISLANDS', 'FRENCH_POLYNESIA', 'COOK_ISLANDS', 'NEW_ZEALAND', 'NORFOLK_ISLAND', 'NEW_ZEALAND', 'AUSTRALIA', 'EAST_TIMOR', 'AUS

In [30]:
for i in list(graph.edges.keys()):
    for j in graph.edges[i]:
        if j not in list(graph.edges.keys()):
            print(j)